<a href="https://colab.research.google.com/github/akbarriki/scraping_tiketcom/blob/main/Scraping_Tiketcom_BestPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import requests, json, time, math, os
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime as dt
from google.colab import drive

#mounting google drive
drive.mount('/content/gdrive', force_remount=True)
project_folder = 'gdrive/MyDrive/scraping_projects/tiketcom'
output_file = 'tiketcom_bestprice.csv'
os.chdir(project_folder)
os.getcwd()

Mounted at /content/gdrive


'/content/gdrive/MyDrive/scraping_projects/tiketcom'

In [2]:
# define execution timestamp
exec_timestamp = dt.now()

# define all required variables
base_url = 'https://www.tiket.com/'

# target list
_data_ = []

num_adult, num_child, num_infant = 1, 0, 0
_ORIGIN_, _ORIGINTYPE_ = 'JKTC', 'CITY'
# _DESTINATION_, _DESTINATIONTYPE_ = 'BTJ', 'AIRPORT'
_CABINCLASS_ = 'ECONOMY'
selecteddate = '2023-04-19'
startdate = '2023-04-01'
enddate = '2023-04-30'

_DESTINATIONLIST_ = ['BTJ','BDO','MLG','SRG','SUB','JOG','YIA','SOC','TKG','BTH',
                     'BKS','DJB','KNO','PDG','PLM','PGK','PKU','TNJ','BPN','BDJ',
                     'PNK','SRI','TRK','UPG','MDC','DPS','LOP','BIK','DJJ','MKQ']

for _, _DESTINATION_ in enumerate(tqdm(_DESTINATIONLIST_)):
  # define the type of destination (i.e., CITY or AIRPORT)
  _DESTINATIONTYPE_ = 'AIRPORT'

  # target url
  url = f'{base_url}ms-gateway/tix-price-summary/priceSummary/priceSummary?adult={num_adult}&child={num_child}&infant={num_infant}&origin={_ORIGIN_}&destination={_DESTINATION_}&startDate={startdate}&endDate={enddate}&currency=IDR&originType={_ORIGINTYPE_}&destinationType={_DESTINATIONTYPE_}&cabinClass={_CABINCLASS_}&flightAggregateType=OW'

  # referer url for headers setup
  referer = f'{base_url}pesawat/search?d={_ORIGIN_}&a={_DESTINATION_}&dType={_ORIGINTYPE_}&aType={_DESTINATIONTYPE_}&date={selecteddate}&adult={num_adult}&child={num_child}&infant={num_infant}&class={_CABINCLASS_.lower()}&flexiFare=false'

  # header config
  header = {
      'Content-Type': 'application/json',
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
      'Referer': referer,
      'X-Country-Code': 'IDN',
      'X-Request-id': '064f288d-65a8-4afb-87a0-a5a0647be197',
      'X-Store-Id': 'TIKETCOM'
  }

  # scraping process
  page = requests.get(url, headers=header)
  response = json.loads(page.text)
  try:
    _data_ += [{'extract_timestamp':exec_timestamp,
                'origin': _ORIGIN_,
                'destination': _DESTINATION_,
                'depart_date':r['date'], 
                'best_price':r['price']} for r in response['data']]
  except:
    print(f'Fail to extract data (destination: {_DESTINATION_})!')


# write/append data to the target csv file
if _data_:
  df = pd.DataFrame(_data_)
  if not os.path.isfile(output_file):
    df.to_csv(output_file, index=False, sep='|')
    print(f'data has been exported to {output_file}')
  else:
    df.to_csv(output_file, index=False, sep='|', mode='a')
    print(f'file {output_file} has been appended with new data')
else:
  print('No data found')

100%|██████████| 30/30 [00:10<00:00,  2.90it/s]

data has been exported to tiketcom_bestprice.csv


In [3]:
pd.read_csv(output_file, sep='|')

,extract_timestamp,origin,destination,depart_date,best_price
0,2022-12-24 11:25:37.656571,JKTC,BTJ,2023-04-25,2310582.0
1,2022-12-24 11:25:37.656571,JKTC,BTJ,2023-04-04,2310582.0
2,2022-12-24 11:25:37.656571,JKTC,BTJ,2023-04-27,2310582.0
3,2022-12-24 11:25:37.656571,JKTC,BTJ,2023-04-29,2310582.0
4,2022-12-24 11:25:37.656571,JKTC,BTJ,2023-04-01,2316313.0
...,...,...,...,...,...
861,2022-12-24 11:25:37.656571,JKTC,MKQ,2023-04-08,4786080.0
862,2022-12-24 11:25:37.656571,JKTC,MKQ,2023-04-12,4786080.0
863,2022-12-24 11:25:37.656571,JKTC,MKQ,2023-04-13,4786080.0
864,2022-12-24 11:25:37.656571,JKTC,MKQ,2023-04-15,4786080.0
